# COVID-19 plots
> Plots with daily updates

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [healthcare, covid]
- image: images/covid_plots/covid_mask.png
- hide: false
- search_exclude: true

In [1]:
#collapse
from typing import Callable, Literal
import warnings
import pandas as pd
import altair as alt

# Data

In [2]:
df = pd.read_csv('https://covidtracking.com/api/states/daily.csv')

# Helpers

In [3]:
#collapse
def create_df(df: pd.DataFrame, 
              filter_f: Callable, 
              state_name: str, 
              period: int = 7*8):
    out_df = (
     df
     .assign(date = lambda d: pd.to_datetime(d['date'], format='%Y%m%d'))
     .loc[filter_f, :]
     .groupby('date')
     .apply(lambda d: pd.Series(dict(
         totalTestResultsIncrease = d['totalTestResultsIncrease'].sum(),
         positiveIncrease = d['positiveIncrease'].sum(),
         death = d['death'].sum(),
         deathIncrease = d['deathIncrease'].sum()
     )))
     .reset_index()
     .assign(positiveRate = lambda d: round(d['positiveIncrease'] / d['totalTestResultsIncrease'], 3))
     .loc[lambda d: d['date'] >= pd.to_datetime('today') - period * pd.Timedelta('1 days'), :]
     .sort_values('date', ascending=False)
     .assign(state = state_name)
     .reset_index(drop=True)
    )
    
    return out_df

In [4]:
#collapse
def create_plot(df: pd.DataFrame, 
                    column: Literal['totalTestResultsIncrease', 
                                    'positiveIncrease', 
                                    'death',
                                    'deathIncrease', 
                                    'positiveRate']):
    the_state = df['state'][0]
    points = alt.Chart(df).mark_point(size=50).encode(
        x='date:T',
        y=f'{column}:Q',
        tooltip=['date', column]
    ).properties(
        title=f'{the_state}: {column}',
        width=800,
        height=400
    )
    line = alt.Chart(df).mark_line(
        color='purple',
        size=2
    ).transform_window(
        rolling_mean=f'mean({column})',
        frame=[0, 3]
    ).encode(
        x='date:T',
        y='rolling_mean:Q'
    )

    return (points + line).interactive()

# US

In [5]:
us = create_df(df, lambda d: d.index > 0, 'US')

In [6]:
create_plot(us, 'positiveIncrease')

alt.LayerChart(...)

In [7]:
create_plot(us, 'positiveRate')

alt.LayerChart(...)

In [8]:
create_plot(us, 'deathIncrease')

alt.LayerChart(...)

# NY

In [9]:
ny = create_df(df, (lambda d: d['state'] == 'NY'), 'New York')

In [ ]:
create_plot(ny, 'positiveIncrease')

In [ ]:
create_plot(ny, 'positiveRate')

In [ ]:
create_plot(ny, 'deathIncrease')

# Non Tri-State

In [ ]:
non_tristate = create_df(df, lambda d: ~d['state'].isin(['CT','NJ','NY']), 'Non Tri-State')

In [ ]:
create_plot(non_tristate, 'positiveIncrease')

In [ ]:
create_plot(non_tristate, 'positiveRate')

In [ ]:
create_plot(non_tristate, 'deathIncrease')